In [0]:
from pyspark.sql import SparkSession

sparkSess = SparkSession.builder.appName('F1Analysis').getOrCreate()

input_df = sparkSess.read.format('csv').option('header',True).load('dbfs:/FileStore/tables/drivers.csv')



In [0]:
%run "./reader_class"

In [0]:
class workFlow:
    def __init__(self):
        pass

    def runner(self):
        driver_data = load_data_source(
            data_type = "csv",
            file_path = 'dbfs:/FileStore/tables/drivers.csv'
        ).get_data_frame()

        driver_data.show()

workflow = workFlow()

In [0]:
input_df = workflow.runner()

+--------+----------+------+----+---------+----------+----------+-----------+--------------------+
|driverId| driverRef|number|code| forename|   surname|       dob|nationality|                 url|
+--------+----------+------+----+---------+----------+----------+-----------+--------------------+
|       1|  hamilton|    44| HAM|    Lewis|  Hamilton|1985-01-07|    British|http://en.wikiped...|
|       2|  heidfeld|    \N| HEI|     Nick|  Heidfeld|1977-05-10|     German|http://en.wikiped...|
|       3|   rosberg|     6| ROS|     Nico|   Rosberg|1985-06-27|     German|http://en.wikiped...|
|       4|    alonso|    14| ALO| Fernando|    Alonso|1981-07-29|    Spanish|http://en.wikiped...|
|       5|kovalainen|    \N| KOV|   Heikki|Kovalainen|1981-10-19|    Finnish|http://en.wikiped...|
|       6|  nakajima|    \N| NAK|   Kazuki|  Nakajima|1985-01-11|   Japanese|http://en.wikiped...|
|       7|  bourdais|    \N| BOU|Sébastien|  Bourdais|1979-02-28|     French|http://en.wikiped...|
|       8|

What does Spark do when we run the cell?

It builds a logical execution plan and creates Jobs, Stages and Tasks based on the computations.     
1. A job in Spark represents some kind of computation and is triggered when an action is called such as 'count()', 'collect()', 'save()'.      
2. Each job is broken down into smaller, manageable units called Stages are created when there is shuffling of data or wide transformations such as 'groupBy()' or 'reduceByKey()'.
3. Tasks are the smallest unit of work in Spark. It represents the computation performed on a single partition of the dataset. 